In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# set up subfolders so we have different locations to store training and test data 
os.mkdir('/kaggle/temp')
os.chdir('/kaggle/temp')
os.mkdir('train')
os.mkdir('valid')
os.mkdir('test')
os.chdir('/kaggle/working')

In [3]:
# label the data path 
data_path = '/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms'

In [4]:
# create a list of images and for each image, create a 2d list of pixels 
for genus in os.listdir(data_path):
    print('genus: ', genus)
    # create path to current genus
    path = f'{data_path}/{genus}'
#     print(path)
    # make list of each file within current genus
    data = os.listdir(path)
#     print('data: ', data)
    # divvy out the images into training, valid, and testing groups
    num_mushrooms = len(data)
    train_mushrooms = int(num_mushrooms * .7)
    train_path = f'/kaggle/temp/train/{genus}'
    
    valid_mushrooms = int(num_mushrooms * .8)
    valid_path = f'/kaggle/temp/valid/{genus}'
    
    test_path = f'/kaggle/temp/test/{genus}'
    
    # change to the training data and create a subfolder per genus
    os.chdir('/kaggle/temp/train')
    os.mkdir(genus)
    # copy over data in the training data range into the new directory 
    for mushroom in range(train_mushrooms):
        original = f'{data_path}/{genus}/{data[mushroom]}'
        print('original: ', original)
        new_mushroom = f'{train_path}/{data[mushroom]}'
        print('new: ', new_mushroom)
        shutil.copyfile(original, new_mushroom)
        
    # change to the validation images and create a subfolder per genus
    os.chdir('/kaggle/temp/valid')
    os.mkdir(genus)
    # copy over data in the validation data range into the new directory
    for mushroom in range(train_mushrooms, valid_mushrooms):
        original = f'{data_path}/{genus}/{data[mushroom]}'
        new_mushroom = f'{valid_path}/{data[mushroom]}'
        shutil.copyfile(original, new_mushroom)
    
    # change the to testing data
    os.chdir('/kaggle/temp/test')
    os.mkdir(genus)
    # copy over data from the testing images and put images in a new dir
    for mushroom in range(valid_mushrooms, num_mushrooms):
        original = f'{data_path}/{genus}/{data[mushroom]}'
        new_mushroom = f'{test_path}/{data[mushroom]}'
        shutil.copyfile(original, new_mushroom)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input


In [6]:
# standardize the batch and image size
image_dim = 320
batch_size = 32

In [7]:
# training data image set 
training_images = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True,
                                         width_shift_range=0.1, height_shift_range=0.1)\
                    .flow_from_directory(directory='/kaggle/temp/train',
                                         target_size=(image_dim, image_dim),
                                         class_mode= 'categorical',
                                         batch_size= batch_size,
                                         shuffle=True)

In [8]:
# validation data image set 
valid_images = ImageDataGenerator(preprocessing_function=preprocess_input)\
                    .flow_from_directory(directory='/kaggle/temp/valid',
                                         target_size=(image_dim, image_dim),
                                         class_mode= 'categorical',
                                         batch_size= batch_size,
                                         shuffle=True)

In [9]:
# test data image set 
test_images = ImageDataGenerator(preprocessing_function=preprocess_input)\
                    .flow_from_directory(directory='/kaggle/temp/test',
                                         target_size=(image_dim, image_dim),
                                         class_mode='categorical',
                                         batch_size=batch_size,
                                         shuffle=True)